In [1]:
import random
from keras.layers import Dense, Activation, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten
from keras.models import Sequential
import numpy as np
import pickle
import json
from tensorflow.keras.optimizers import SGD
import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='loss', min_delta=0,
                          patience=150, verbose=1, restore_best_weights=True)

lemmatizer = WordNetLemmatizer()

In [3]:
def numberToLetter(w):
    new_w = []
    for i in w:
        new_i = ""
        for j in range(0,len(i)):
            if i[j]=='1':
                new_i+='one'
            elif i[j]=='2':
                new_i+='two'
            elif i[j]=='3':
                new_i+='three'
            elif i[j]=='4':
                new_i+='four'
            elif i[j]=='5':
                new_i+='five'
            elif i[j]=='6':
                new_i+='six'
            elif i[j]=='7':
                new_i+='seven'
            elif i[j]=='8':
                new_i+='eight'
            elif i[j]=='9':
                new_i+='nine'
            elif i[j]=='0':
                new_i+='zero'
            else:
                new_i+=i[j]
        new_w.append(new_i)
    return(new_w)
        

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("merged_dataset_intents.json").read()
intents = json.loads(data_file)
nltk.download('wordnet')

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # tokenize each word
        w = nltk.word_tokenize(pattern)
        # print(numberToLetter(w))
        words.extend(numberToLetter(w))
        # add documents in the corpus
        documents.append((numberToLetter(w), intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Yash\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
numberToLetter(['The','cake','was','2','kg','heavy'])

['The', 'cake', 'was', 'two', 'kg', 'heavy']

In [6]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower())
         for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)


1189 documents
1134 classes ['0000053', '0000054', '0000055', '0000056', '0000057', '0000058', '0000059', '0000060', '0000061', '0000062', '0000063', '0000064', '0000065', '0000066', '0000067', '0000068', '0000069', '0000070', '0000071', '0000072', '0000073', '0000074', '0000075', '0000076', '0000077', '0000078', '0000079', '0000080', '0000081', '0000082', '0000083', '0000084', '0000085', '0000086', '0000087', '0000088', '0000089', '0000090', '0000091', '0000092', '0000093', '0000094', '0000095', '0000096', '0000097', '0000098', '0000099', '0000100', '0000101', '0000102', '0000103', '0000104', '00001043', '0000105', '00001051', '00001052', '00001053', '00001054', '00001055', '0000106', '0000107', '00001075', '00001076', '00001077', '00001078', '00001079', '0000108', '0000109', '0000110', '0000112', '0000113', '0000114', '0000115', '0000116', '0000117', '0000118', '0000119', '0000120', '0000121', '0000122', '0000123', '0000124', '0000125', '0000126', '0000127', '0000128', '0000129', '00

In [7]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [8]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(
        word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

Training data created


C:\Users\Yash\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [9]:
# Inital model used in prototype build

In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])

C:\Users\Yash\AppData\Roaming\Python\Python37\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               122496    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1134)              73710     
                                                                 
Total params: 204,462
Trainable params: 204,462
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Final model

In [12]:
model2 = Sequential()
model2.add(Dense(256,  activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(128,  activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])

In [13]:
X_train=np.array(train_x)
y_train=np.array(train_y)

In [14]:
# fitting and saving the model
hist2 = model2.fit(X_train, y_train,
                 epochs=1500, batch_size=64, verbose=1,
                 callbacks=[earlystop]
                 )  # batch_size=25 1500

Epoch 1/1500
19/19 [==============================] - 1s 9ms/step - loss: 7.0352 - accuracy: 0.0000e+00
Epoch 2/1500
19/19 [==============================] - 0s 8ms/step - loss: 7.0347 - accuracy: 0.0000e+00
Epoch 3/1500
19/19 [==============================] - 0s 8ms/step - loss: 7.0344 - accuracy: 0.0025
Epoch 4/1500
19/19 [==============================] - 0s 8ms/step - loss: 7.0336 - accuracy: 0.0025
Epoch 5/1500
19/19 [==============================] - 0s 8ms/step - loss: 7.0322 - accuracy: 0.0042
Epoch 6/1500
19/19 [==============================] - 0s 8ms/step - loss: 7.0316 - accuracy: 0.0034
Epoch 7/1500
19/19 [==============================] - 0s 14ms/step - loss: 7.0317 - accuracy: 0.0059
Epoch 8/1500
19/19 [==============================] - 0s 14ms/step - loss: 7.0327 - accuracy: 0.0042
Epoch 9/1500
19/19 [==============================] - 0s 9ms/step - loss: 7.0318 - accuracy: 0.0050
Epoch 10/1500
19/19 [==============================] - 0s 8ms/step - loss: 7.0293 - accura

19/19 [==============================] - 0s 8ms/step - loss: 3.2267 - accuracy: 0.2145
Epoch 162/1500
19/19 [==============================] - 0s 8ms/step - loss: 3.2801 - accuracy: 0.1842
Epoch 163/1500
19/19 [==============================] - 0s 8ms/step - loss: 3.1249 - accuracy: 0.2086
Epoch 164/1500
19/19 [==============================] - 0s 11ms/step - loss: 3.0855 - accuracy: 0.2170
Epoch 165/1500
19/19 [==============================] - 0s 13ms/step - loss: 3.0545 - accuracy: 0.2288
Epoch 166/1500
19/19 [==============================] - 0s 13ms/step - loss: 3.0355 - accuracy: 0.2170
Epoch 167/1500
19/19 [==============================] - 0s 14ms/step - loss: 2.9812 - accuracy: 0.2254
Epoch 168/1500
19/19 [==============================] - 0s 13ms/step - loss: 2.9550 - accuracy: 0.2523
Epoch 169/1500
19/19 [==============================] - 0s 12ms/step - loss: 2.9804 - accuracy: 0.2405
Epoch 170/1500
19/19 [==============================] - 0s 8ms/step - loss: 2.9669 - accura

19/19 [==============================] - 0s 16ms/step - loss: 1.4757 - accuracy: 0.5408
Epoch 242/1500
19/19 [==============================] - 0s 15ms/step - loss: 1.5037 - accuracy: 0.5509
Epoch 243/1500
19/19 [==============================] - 0s 17ms/step - loss: 1.5026 - accuracy: 0.5383
Epoch 244/1500
19/19 [==============================] - 0s 15ms/step - loss: 1.4550 - accuracy: 0.5610
Epoch 245/1500
19/19 [==============================] - 0s 16ms/step - loss: 1.4960 - accuracy: 0.5542
Epoch 246/1500
19/19 [==============================] - 0s 17ms/step - loss: 1.4447 - accuracy: 0.5702
Epoch 247/1500
19/19 [==============================] - 0s 9ms/step - loss: 1.4858 - accuracy: 0.5568: 0s - loss: 1.4115 - accuracy: 0.
Epoch 248/1500
19/19 [==============================] - 0s 8ms/step - loss: 1.4102 - accuracy: 0.5778
Epoch 249/1500
19/19 [==============================] - 0s 13ms/step - loss: 1.4880 - accuracy: 0.5408
Epoch 250/1500
19/19 [==============================] - 

19/19 [==============================] - 0s 9ms/step - loss: 0.7396 - accuracy: 0.7620
Epoch 401/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.7463 - accuracy: 0.7696
Epoch 402/1500
19/19 [==============================] - 0s 9ms/step - loss: 0.7350 - accuracy: 0.7645
Epoch 403/1500
19/19 [==============================] - 0s 16ms/step - loss: 0.7944 - accuracy: 0.7494
Epoch 404/1500
19/19 [==============================] - 0s 14ms/step - loss: 0.7421 - accuracy: 0.7754
Epoch 405/1500
19/19 [==============================] - 0s 14ms/step - loss: 0.7083 - accuracy: 0.7729
Epoch 406/1500
19/19 [==============================] - 0s 14ms/step - loss: 0.7104 - accuracy: 0.7687
Epoch 407/1500
19/19 [==============================] - 0s 9ms/step - loss: 0.7681 - accuracy: 0.7578
Epoch 408/1500
19/19 [==============================] - 0s 11ms/step - loss: 0.7336 - accuracy: 0.7628
Epoch 409/1500
19/19 [==============================] - 0s 14ms/step - loss: 0.7100 - accura

Epoch 480/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.6011 - accuracy: 0.7990
Epoch 481/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.6208 - accuracy: 0.7864
Epoch 482/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.6263 - accuracy: 0.7998
Epoch 483/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.6133 - accuracy: 0.7906
Epoch 484/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.6344 - accuracy: 0.7763
Epoch 485/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.6556 - accuracy: 0.7712
Epoch 486/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.6386 - accuracy: 0.8007
Epoch 487/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.6170 - accuracy: 0.7973
Epoch 488/1500
19/19 [==============================] - 0s 10ms/step - loss: 0.6448 - accuracy: 0.7872
Epoch 489/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.586

...

19/19 [==============================] - 0s 8ms/step - loss: 0.3726 - accuracy: 0.8562
Epoch 879/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3909 - accuracy: 0.8553
Epoch 880/1500
19/19 [==============================] - 0s 9ms/step - loss: 0.4048 - accuracy: 0.8486
Epoch 881/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3712 - accuracy: 0.8612
Epoch 882/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3649 - accuracy: 0.8570
Epoch 883/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3679 - accuracy: 0.8646
Epoch 884/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3915 - accuracy: 0.8646
Epoch 885/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3651 - accuracy: 0.8797
Epoch 886/1500
19/19 [==============================] - 0s 9ms/step - loss: 0.3698 - accuracy: 0.8646
Epoch 887/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3889 - accuracy: 0.

19/19 [==============================] - 0s 7ms/step - loss: 0.3565 - accuracy: 0.8730
Epoch 1038/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3406 - accuracy: 0.8839
Epoch 1039/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3794 - accuracy: 0.8612
Epoch 1040/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.3476 - accuracy: 0.8713
Epoch 1041/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3617 - accuracy: 0.8705
Epoch 1042/1500
19/19 [==============================] - 0s 16ms/step - loss: 0.3762 - accuracy: 0.8537
Epoch 1043/1500
19/19 [==============================] - 0s 16ms/step - loss: 0.3650 - accuracy: 0.8789
Epoch 1044/1500
19/19 [==============================] - 0s 15ms/step - loss: 0.3227 - accuracy: 0.8814
Epoch 1045/1500
19/19 [==============================] - 0s 13ms/step - loss: 0.3488 - accuracy: 0.8680
Epoch 1046/1500
19/19 [==============================] - 0s 14ms/step - loss: 0.3566 

Epoch 1116/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.3501 - accuracy: 0.8755
Epoch 1117/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.3456 - accuracy: 0.8764
Epoch 1118/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3457 - accuracy: 0.8722
Epoch 1119/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3300 - accuracy: 0.8755
Epoch 1120/1500
19/19 [==============================] - 0s 9ms/step - loss: 0.3393 - accuracy: 0.8730
Epoch 1121/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.3335 - accuracy: 0.8797
Epoch 1122/1500
19/19 [==============================] - 0s 6ms/step - loss: 0.3407 - accuracy: 0.8797
Epoch 1123/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.3305 - accuracy: 0.8780
Epoch 1124/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.3459 - accuracy: 0.8696
Epoch 1125/1500
19/19 [==============================] - 0s 8ms/step - lo

19/19 [==============================] - 0s 8ms/step - loss: 0.3575 - accuracy: 0.8696
Epoch 1275/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3029 - accuracy: 0.8831
Epoch 1276/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3005 - accuracy: 0.8839
Epoch 1277/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3107 - accuracy: 0.8823
Epoch 1278/1500
19/19 [==============================] - 0s 11ms/step - loss: 0.2871 - accuracy: 0.8890
Epoch 1279/1500
19/19 [==============================] - 0s 10ms/step - loss: 0.3208 - accuracy: 0.8755
Epoch 1280/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.2892 - accuracy: 0.9024
Epoch 1281/1500
19/19 [==============================] - 0s 9ms/step - loss: 0.3230 - accuracy: 0.8747
Epoch 1282/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3265 - accuracy: 0.8823
Epoch 1283/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3112 - a

19/19 [==============================] - 0s 7ms/step - loss: 0.2713 - accuracy: 0.8949
Epoch 1433/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3377 - accuracy: 0.8705
Epoch 1434/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.2890 - accuracy: 0.8839
Epoch 1435/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.3356 - accuracy: 0.8722
Epoch 1436/1500
19/19 [==============================] - 0s 9ms/step - loss: 0.2886 - accuracy: 0.8848
Epoch 1437/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3046 - accuracy: 0.8755
Epoch 1438/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.2791 - accuracy: 0.8865
Epoch 1439/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.2797 - accuracy: 0.8932
Epoch 1440/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.2837 - accuracy: 0.8966
Epoch 1441/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.3114 - acc

In [15]:
# Further training model 2
hist3 = model2.fit(X_train, y_train,
                 epochs=1500, batch_size=64, verbose=1,
                 callbacks=[earlystop]
                 ) 

Epoch 1/1500
19/19 [==============================] - 0s 9ms/step - loss: 0.2533 - accuracy: 0.9033
Epoch 2/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.2967 - accuracy: 0.8823
Epoch 3/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3260 - accuracy: 0.8856
Epoch 4/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.2864 - accuracy: 0.8848
Epoch 5/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3150 - accuracy: 0.8764
Epoch 6/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3465 - accuracy: 0.8764
Epoch 7/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.2973 - accuracy: 0.8907
Epoch 8/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3095 - accuracy: 0.8848
Epoch 9/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.2917 - accuracy: 0.8823
Epoch 10/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.2727 - accuracy: 0.8966

19/19 [==============================] - 0s 9ms/step - loss: 0.2708 - accuracy: 0.8890
Epoch 83/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.2811 - accuracy: 0.8923
Epoch 84/1500
19/19 [==============================] - 0s 7ms/step - loss: 0.3048 - accuracy: 0.8865
Epoch 85/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3002 - accuracy: 0.8797
Epoch 86/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.2340 - accuracy: 0.9050
Epoch 87/1500
19/19 [==============================] - 0s 9ms/step - loss: 0.2851 - accuracy: 0.8789
Epoch 88/1500
19/19 [==============================] - 0s 9ms/step - loss: 0.2895 - accuracy: 0.8856
Epoch 89/1500
19/19 [==============================] - 0s 8ms/step - loss: 0.3000 - accuracy: 0.8806
Epoch 90/1500
19/19 [==============================] - 0s 10ms/step - loss: 0.3066 - accuracy: 0.8865
Epoch 91/1500
19/19 [==============================] - 0s 9ms/step - loss: 0.2673 - accuracy: 0.8890
Epo

In [16]:
# Model has 91.02% accuracy
# Saving accuracy
model2.save('chatbot_model.h5', hist3)

In [ ]:
#Testing out the model

In [4]:
import numpy as np
import pickle
import random
import json
from keras.models import load_model
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
model = load_model('chatbot_model.h5')
intents = json.loads(open('merged_dataset_intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [5]:
def numberToLetter(w):
    new_w = []
    for i in w:
        new_i = ""
        for j in range(0,len(i)):
            if i[j]=='1':
                new_i+='one'
            elif i[j]=='2':
                new_i+='two'
            elif i[j]=='3':
                new_i+='three'
            elif i[j]=='4':
                new_i+='four'
            elif i[j]=='5':
                new_i+='five'
            elif i[j]=='6':
                new_i+='six'
            elif i[j]=='7':
                new_i+='seven'
            elif i[j]=='8':
                new_i+='eight'
            elif i[j]=='9':
                new_i+='nine'
            elif i[j]=='0':
                new_i+='zero'
            else:
                new_i+=i[j]
        new_w.append(new_i)
    return(new_w)
        

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # change numbers to words
    sentence_words = numberToLetter(sentence_words)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(
        word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence


def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    # print("Result is ---->", res)
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result


# intents_Mental_Health_FAQ.json 0.4 confidence
# dialog_intents.json 0.3 confidence
# depression_chatbot_intents.json 0.6 confidence
# chatbot_chitchat_intents.json 0.65 confidence

def chatbot_response(msg):
    ints = predict_class(msg, model)
    # 79% confidence
    # print(msg, float(ints[0]['probability']))
    print(msg, ints)
    try:
        if(float(ints[0]['probability']) > 0.4):  # 79
            print("in accept ")
            res = getResponse(ints, intents)
        else:
            print("in reject ")
            res = "I didnt get that"
    except:
        print("Exception")
        res = "I didnt get that"
    return res

In [6]:
print("What is mental health", chatbot_response("What is mental health"))

What is mental health [{'intent': '7728364', 'probability': '0.9999156'}]
in accept 
What is mental health We all have mental health which is made up of our beliefs, thoughts, feelings and behaviours.


In [67]:
print("how can i recover from mental illness",
      chatbot_response("how can i recover from mental illness"))

how can i recover from mental illness [{'intent': '7657263', 'probability': '0.89542395'}]
in accept 
how can i recover from mental illness When healing from mental illness, early identification and treatment are of vital importance. Based on the nature of the illness, there are a range of effective treatments available. For any type of treatment, it is essential that the person affected is proactive and fully engaged in their own recovery process.
Many people with mental illnesses who are diagnosed and treated respond well, although some might experience a return of symptoms. Even in such cases, with careful monitoring and management of the disorder, it is still quite possible to live a fulfilled and productive life.


In [68]:
print("Its a great day", chatbot_response("Its a great day"))
print("You look cute", chatbot_response("You look cute"))
print("Had dinner?", chatbot_response("Had dinner?"))

Its a great day [{'intent': '0000880', 'probability': '0.76239735'}]
in accept 
Its a great day yes, i do.
You look cute [{'intent': '0001112', 'probability': '0.32012266'}, {'intent': '0001114', 'probability': '0.3029895'}]
in reject 
You look cute I didnt get that
Had dinner? []
Exception
Had dinner? I didnt get that


In [69]:
print("I hate loosing", chatbot_response("I hate loosing"))
print("What is depression", chatbot_response("What is depression"))
print("Is it normal to be depressed",
      chatbot_response("Is it normal to be depressed"))


I hate loosing [{'intent': 'i hate myself!', 'probability': '0.9523862'}]
in accept 
I hate loosing when god was creating the universe he gave humans the power to think and the contemplate, so that we can understand the world and make it a better place. god did not give you brains to hate yourself, it wuld be wiser to love yourself and show god that you are serving the purpose for which you were sent.
What is depression [{'intent': 'what is depression?', 'probability': '0.9999994'}]
in accept 
What is depression depression is a common and serious medical illness that negatively affects how you feel, the way you think and how you act.fortunately,it is also treatable.depression causes feelings of sadness and/or a loss of interest in activities you once enjoyed. it can lead to a variety of emotional and physical problems and can decrease your ability to function at work and at home.
Is it normal to be depressed [{'intent': '0000152', 'probability': '0.9997063'}]
in accept 
Is it normal to

In [70]:
print("Let us gossip", chatbot_response("Let us gossip"))
print("Do you have a laptop ?", chatbot_response("Do you have a laptop ?"))
print("Will it rain today", chatbot_response("Will it rain today"))
print("I m bored", chatbot_response("I m bored"))

Let us gossip [{'intent': '1000018', 'probability': '0.99371076'}]
in accept 
Let us gossip John said I tend to say the same things repeatedly regardless of what he is saying.
Do you have a laptop ? [{'intent': '0000891', 'probability': '0.889312'}]
in accept 
Do you have a laptop ? no, i don't. do you?
Will it rain today [{'intent': '0000174', 'probability': '0.47737944'}]
in accept 
Will it rain today i think it will be.
I m bored [{'intent': '0000946', 'probability': '0.4463746'}, {'intent': 'i am lonely!', 'probability': '0.30181044'}]
in accept 
I m bored what's on tv?
